In [1]:
import os
import pdf2image
from PIL import Image
import pytesseract
import difflib
import re
import pandas as pd
import sys
sys.path.append('../')
from helper import *

In [2]:
DATA_PATH = '../../data/'
STATE = 'sikkim/'

PARSE_DATA_PAGES = "../../parseData/images/"+STATE
create_path(PARSE_DATA_PAGES)

PARSE_DATA_BLOCKS = "../../parseData/blocks/"+STATE
create_path(PARSE_DATA_BLOCKS)

PARSE_DATA_CSVS = "../../parseData/csvs/"+STATE
create_path(PARSE_DATA_CSVS)

COLUMNS = ["number","id", "elector_name", "father_or_husband_name", "relationship", "house_no", "age", "sex", "ac_name", "parl_constituency", "part_no", "year", "state", "filename", "main_town", "police_station", "mandal", "revenue_division", "district", "pin_code", "polling_station_name", "polling_station_address", "net_electors_male", "net_electors_female", "net_electors_third_gender", "net_electors_total"]

state_pdfs_path = DATA_PATH+STATE
state_pdfs_files = os.listdir(state_pdfs_path)

In [3]:
sort_nicely(state_pdfs_files)
state_pdfs_files

['01_001.pdf',
 '01_002.pdf',
 '01_003.pdf',
 '01_004.pdf',
 '01_005.pdf',
 '01_006.pdf',
 '01_007.pdf',
 '01_008.pdf',
 '01_009.pdf',
 '01_010.pdf',
 '01_011.pdf',
 '01_012.pdf',
 '01_013.pdf',
 '01_014.pdf',
 '01_015.pdf',
 '01_016.pdf',
 '01_017.pdf',
 '01_018.pdf',
 '01_019.pdf',
 '02_001.pdf',
 '02_002.pdf',
 '02_003.pdf',
 '02_004.pdf',
 '02_005.pdf',
 '02_006.pdf',
 '02_007.pdf',
 '02_008.pdf',
 '02_009.pdf',
 '02_010.pdf',
 '02_011.pdf',
 '02_012.pdf',
 '02_013.pdf',
 '02_014.pdf',
 '02_015.pdf',
 '02_016.pdf',
 '03_001.pdf',
 '03_002.pdf',
 '03_003.pdf',
 '03_004.pdf',
 '03_005.pdf',
 '03_006.pdf',
 '03_007.pdf',
 '03_008.pdf',
 '03_009.pdf',
 '03_010.pdf',
 '03_011.pdf',
 '03_012.pdf',
 '03_013.pdf',
 '03_014.pdf',
 '03_015.pdf',
 '03_016.pdf',
 '03_017.pdf',
 '03_018.pdf',
 '03_019.pdf',
 '03_020.pdf',
 '03_021.pdf',
 '03_022.pdf',
 '04_001.pdf',
 '04_002.pdf',
 '04_003.pdf',
 '04_004.pdf',
 '04_005.pdf',
 '04_006.pdf',
 '04_007.pdf',
 '04_008.pdf',
 '04_009.pdf',
 '04_010.p

In [45]:
def split_data(data):
    seps = [":","-",">","°"]
    
    for s in seps:
        if s in data:
            break

    data = data.split(s)
    data = [ i for i in data if i.strip()!='']
    if len(data)>1:
        data = data[1].strip()
        return data
    else:
        data = ""
        return data
    
def extract_first_page_details(path):

    img = Image.open(path)
    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    a,b,c,d = 162,332,3067,332 # ac name and parl
    crop_img = crop_section(a,b,c,d,img)
    
    crop_ac_path = crop_path+"ac.jpg"
    crop_img.save(crop_ac_path)

    text = (pytesseract.image_to_string(crop_ac_path, config='--psm 6', lang='eng')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    ac_name, parl_constituency = "",""
    
    
    if len(text) == 2:
        ac_name = split_data(text[0])
        
        parl_constituency = text[1].split("(ies)")
        if len(parl_constituency)>1:
            parl_constituency = parl_constituency[1].strip()
        
    elif len(text)==3:
        ac_name = split_data(text[0]) + " "+ text[1]
    
        parl_constituency = text[2].split("(ies)")
        if len(parl_constituency)>1:
            parl_constituency = parl_constituency[1].strip()
    
    a,b,c,d = 160,3333,1776,700 # police name name and address
    crop_img = crop_section(a,b,c,d,img)
    
    crop_police_path = crop_path+"police.jpg"
    crop_img.save(crop_police_path)

    text = (pytesseract.image_to_string(crop_police_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    polling_station_name, polling_station_address = "",""
    
    if len(text) == 4:
        polling_station_name, polling_station_address = text[1],text[3]
    elif len(text)>=5:
        polling_station_name, polling_station_address = text[1],text[3]+" "+text[4]
    
        
    a,b,c,d = 3296,394,600,121 # part no
    crop_img = crop_section(a,b,c,d,img)
    
    crop_part_path = crop_path+"part.jpg"
    crop_img.save(crop_part_path)

    text = (pytesseract.image_to_string(crop_part_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = re.findall(r'\d+', text)
    
    if len(text)>0:
        part_no = text[0]
    else:
        part_no = ""
        
        
    a,b,c,d = 1560,4785,2271,162  # stats for male and female
    crop_img = crop_section(a,b,c,d,img)

    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    crop_stat_path = crop_path+"stat.jpg"
    crop_img.save(crop_stat_path)
    
    a_n,b_n,c_n,d_n = extract_4_numbers(crop_stat_path)
    
        
    a,b,c,d = 2150,2097,1682,1042  # mandal block
    crop_img = crop_section(a,b,c,d,img)
    
    crop_det_path = crop_path+"det.jpg"
    crop_img.save(crop_det_path)

    text = (pytesseract.image_to_string(crop_det_path, config='--psm 6', lang='eng+hin')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    main_town,revenue_division,police_station,mandal,district,pin_code = "","","","","",""

    
    if len(text) == 8:
        
        main_town = split_data(text[0])
        revenue_division = split_data(text[5])
        police_station = split_data(text[4])
        mandal = split_data(text[2])
        district = split_data(text[6])
        pin_code =split_data(text[7])

   
    return [ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,a_n,b_n,c_n,d_n]



In [4]:
def extract_4_numbers(crop_stat_path):
    
    text = (pytesseract.image_to_string(crop_stat_path, config='--psm 11', lang='eng')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = re.findall(r'\d+', text)

    if len(text)==4:
        if int(text[0]) + int(text[1]) == int(text[2]):
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[2]
        elif int(text[0]) + int(text[1]) == int(text[3]):
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[3]
        else:
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],text[2],text[3]
    elif len(text) == 3 and int(text[2])>=int(text[1]) and int(text[2])>=int(text[0]):
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[2]
    elif len(text) == 2 and int(text[0])*2-100<int(text[1]):
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],int(text[1])-int(text[0]),"0",text[1]
    else:
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = "","","",""
    
    return net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total

In [60]:
def extract_name(name):
    
    row = name.split(":")
    if len(row)!=2:
        return ""
    else:
        return row[1].strip()
    
def extract_vid(v_id):
    row = v_id.split(" ")
    if len(row)>=2:
        number = re.findall(r'\d+', row[0].strip())
        if len(number)>0:
            return number[0],row[-1]
        else:
            return "",row[-1]
    
    if len(row)==1:
        return "", row[0]
    else:
        return "",v_id

def extract_house_no(house_no):
    row = house_no.split(":")
    if len(row)==2:
        house_no = re.findall(r'\d+', row[1].strip())
        if len(house_no)>0:
            return house_no[0]
        else:
            return ""
    else:
        house_no = re.findall(r'\d+', row[0].strip())
        if len(house_no)>0:
            return house_no[0]
        else:
            return ""
    
def extract_age_gender(age_gender):
    
    age = re.findall(r'\d+', age_gender.strip())
    if len(age)>0:
        age = age[0]
    else:
        age = ""
    
    gender = ''
    
    if 'Female' in age_gender.strip():
        gender = 'Female'
    elif 'Male' in age_gender.strip():
        gender = "Male"
    else:
        gender = ""
        

    return age, gender

def extract_rel_name(rel_name):
    name = split_data(rel_name)
    type = extract_rel_type(rel_name)
    return name, type
    
def extract_rel_type(rel_type):
    line = rel_type
    if line.startswith("Husband") :
        rel_type = 'husband'
    elif line.startswith("Father"):
        rel_type = 'father'
    elif line.startswith("Mother") :
        rel_type = 'mother'
    else:
        rel_type = ""
    
    return rel_type 


def extract_details_from_block(block):
    
    if len(block)==5:
    
        v_id = block[0]
        name = block[1]
        rel_name = block[2]
        house_no = block[3]
        age_gender = block[4]
        
    
    name = split_data(name)
    rel_name,rel_type = extract_rel_name(rel_name)
    house_no = extract_house_no(house_no)
    age, gender = extract_age_gender(age_gender)
    number,voter_id = extract_vid(v_id)
    
    
    return [name,rel_name,rel_type,house_no,age,gender,voter_id,number]

def arrange_columns(first_page_list,block_list,filename):
    
    year = 2018
    state = 'sikkim'
    
    ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = first_page_list
    name,rel_name,rel_type,house_no,age,gender,voter_id,number = block_list
    
    final_list = [number,voter_id,name,rel_name,rel_type,house_no,age,gender,ac_name,
                 parl_constituency,part_no,year,state,filename,main_town,police_station,mandal,
                 revenue_division,district,pin_code,polling_station_name,polling_station_address,
                 net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total]
    
    
    return final_list

In [61]:
def generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page, height):
    
    img = Image.open(page_full_path)
    
    def generate(intial_width,a,b,gap):
        count = 0
        crop_width = 1285
        crop_height = 484

        for col in range(1,11):

            for row in range(1,4):
                c = a+crop_width
                d = b+crop_height
                area = (a, b, c, d)
                cropped_img = img.crop(area)
                count = count+1
                cropped_img.save(page_blocks_path+str(count)+".jpg")

                a = c

            a = intial_width
            b = b+crop_height+gap
    
    intial_width = 130
    generate(intial_width,intial_width,height,17)


In [63]:
def check_amend(page_full_path):
    
    img = Image.open(page_full_path)

    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    a,b,c,d = 241,985,1548,179  # amend
    crop_img = crop_section(a,b,c,d,img)
    
    crop_amend_path = crop_path+"amend.jpg"
    crop_img.save(crop_amend_path)

    text = (pytesseract.image_to_string(crop_amend_path, config='--psm 6', lang='eng')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    
    if "NUMBER OF ELECTORS" in text:    
        return True
    else:
        return False
    

In [65]:
for pdf_file_name in ['01_001.pdf']:
# for pdf_file_name in state_pdfs_files:

    print(pdf_file_name)
    
    if pdf_file_name == ".DS_Store":
        continue
    
    if not pdf_file_name.endswith(".pdf"):
        continue
    
    #create images,blocks and csvs paths for each file
    pdf_file_name_without_ext = pdf_file_name.split('.pdf')[0]
    input_pdf_images_path = PARSE_DATA_PAGES+pdf_file_name_without_ext+"/"
    create_path(input_pdf_images_path)
    
    input_images_blocks_path = PARSE_DATA_BLOCKS+pdf_file_name_without_ext+"/"
    create_path(input_images_blocks_path)
    
    #convert pdf into bunch of images
#     pdf_2_images_list = pdf_to_img(state_pdfs_path+pdf_file_name, input_pdf_images_path,dpi=500)


    #sort pages for looping
    input_images = os.listdir(input_pdf_images_path)
    sort_nicely(input_images)
    
    #empty intial data
    df = pd.DataFrame(columns = COLUMNS)
    order_problem = []
        
    amend_page = False
    
           
    #for each page, parse the data
    for page in input_images:
        
        page_full_path = input_pdf_images_path+page
        
        #extract first page content
        if page == '1.jpg':
            first_page_list = extract_first_page_details(page_full_path)
            continue

        #ingnore 2nd page and last page
        if input_images[-1] == page:
            continue
        
        if os.path.exists(PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv"):
            print(pdf_file_name_without_ext+".csv", "already exists")
            break

        #loop from 3 page onwards
        if page.endswith('.jpg'):
            
            
            if check_amend(page_full_path):
                print("Amend page end ", page)
                break
            
            final_invidual_blocks = []
            blocks_path = input_images_blocks_path+"blocks/"
            create_path(blocks_path)

            page_idx = page.split(".jpg")[0] + "/"
            page_blocks_path = blocks_path+page_idx
            create_path(page_blocks_path)
                        
            if page == '2.jpg':
                generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page, 505)
                
            else:
                generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page, 344)
                            
            
            sorted_blocks = os.listdir(page_blocks_path)
            sort_nicely(sorted_blocks)
            
            for jpg_file in sorted_blocks:
                
                if jpg_file.endswith('.jpg') :

                    text = (pytesseract.image_to_string(page_blocks_path+jpg_file, config='--psm 6', lang='eng')) #config='--psm 4' config='-c preserve_interword_spaces=1'
                    params_list = text.split('\n')
                    new_params_list = [ i for i in params_list if i!='' and i!='\x0c']
                    
                    if len(new_params_list)==5:
                        final_invidual_blocks.append(new_params_list)
                    else:
                        order_problem.append((page, jpg_file,new_params_list))

        #put the data into dataframe
        for block in final_invidual_blocks:
            block_list = extract_details_from_block(block)

            final_list = arrange_columns(first_page_list,block_list,pdf_file_name_without_ext)

            df_length = len(df)
            df.loc[df_length] = final_list
            
        print("page done", page)
                
    #save the dataframe(pdf) data into csv
    save_to_csv(df,PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv")
    print("CSV saved")
    print()


01_001.pdf
HANG MOT) LIMBOO (SUBBA)
PHIP LALL LIMBOO (SUBBA) father

4 Female
1 RBSO0G0G32

AITA HANG SUBBA
BUDHA SINGH LIMBOO father

30 Male
 RBS0067298

TANCHOK HANG SUBBA (LIMBO)
PURNALALL SUBBA (LIMBOOD) father

23 Male
 RBSO060582

PHIP BOR. SUBBA
BIRMAN SUBBA father
7
43 Male
4 FKDO069385

NEERA PRADHAN
ASHOK KUMAR LIMBOO husband
100
28 Female
 RBSOO6O491

BUDH! MAA SUBBA
SUNJETHA SUBBA father
100
64 Male
 FKDOO76851

SUN MAA SUBBA
BUCH! MAN SUBBA husband
100
[] Female
 FKDOOTSS44

SUR MOT! SUBBA
PURNA KUMAR SUBBA husband
10
36 Female
 FKDO058578

PURNA KR SUBBA
BUDH! MAN SUBBA father
100
2 Male
8 FkD00s8321

ASHOK KUMAR SUBBA
BUDHI MAN SUBBA father
100
31 Male
10 FKDOO67785

SUN RANI SUBBA
BUDDH! MAN SUBBA father
100
29 Female
 FKD0108233

BINOD SHEKHAR CHHETRI
YANU RAJ CHHETRI father
107
46 Male
 FKDOOT7E794

SANICHA HANG LIMBOO
THOPJEET LIMBOO father
102
26 Male
 RBS0052696

THOPJIT SUBBA
BATMAREY SUBBA/LIMBOO father
102
72 Male
 FKDO076471

SUK RAJ SUBBA
THOPJIT SUBBA father

HAN GMA LIMBOO
KAMSAMAN LIMBOO father
7
2 Female
136 RBSO008S81

PHIP HANGMA LIMBOOD
LT. RAMS AKAN LIMEOO father

28 Female
 RBSOO52662

SUE ALAN! SUBBA
MON BOR. SUBBA father

45 Female
 FKDO108217

HAN GMA SUBBA
KAMS.AMAN SUBBA father

4 Female
139 FKD0108530

ANITA SUBBA
KAMSA MAN SUBBA father

237 Female
 FKDO108548

- PHIPA HANG LIMBOD
BIR BAHADUR LIMBO father
88
22 Male
 RBS0073973

BIR BOR. SUBBA
SUN KANCHA SUBBA father
8
68 Male
142 FKDOO77081

BISHNU MAYA SUBBA
BIR BOR. SUBBA husband
88
457 Female
 FKD0077032

SANCHA RANI SUBBA
 father

30 Female
 FKDO058057

SUK BIR SUBBA
BIR BAHADUR SUBBA father
8
30 Male
145 RBSO041616

DHAN BIR SUBBA
BIR BAHADUR SUBBA father
88
36 Male
 RBS0041707

TUMBO HANG SUBBA
BIR BAHADUR SUBBA father
8
[] Male
 RBS0073585

BUDHA SINGH SUBBA
DHAN KANCHA SUBBA father
0
5 
 FKDOOTTS09

PURNI MAYA SUBBA
BUDHA SINGH SUBBA husband
90
453 Female
 FKDOOT5051

SANGAL! SUBBA
BUDHA SINGH SUBBA father
80
33 Female
 FKDO108191

MAITA BIR SUBBA (LIMBOO)
BUDHA SINGH

CHUNG PHUT] LEPCHA
RAMSINGH LEPCHA husband
1561
[] Female
267 FKDOOS6788

GONDAY LEPCHA
KIT TSHERING LEPCHA father
156
5 Male
260 FKDOOTOGE4

PEM TSHERING LEPCHA
- PINTSO LEPCHA father
161
43 Male
 RBSO0G0814

YUKIT LEPCHA
PEM TSHERING LEPCHA husband
156
36 Female
 FKDO0S8214

page done 11.jpg
KARMA WANGCHUK LEPCHA
PHURBA TSH. LEPCHA father
1567
29 Male
 FRDOOTS?54

AMGIM LEPCHA
PINTSO LEPCHA father
1581
26 Female
 RBSOO52894

KUMARI LEPCHA
TSHERING LEPCHA husband
16
64 Female
2473 FKDO076620

LABZANG LEPCHA
TSHERING LEPCHA father
16
47 Male
 757

TASH! TSHERING LEPCHA
PASSANG LEPCHA father
16
32 Male
 RBS0052852

- TASH! LEPCHA
TSHERIMNG LEPCHA father
16
2 Male
 RBSO004150

TSHERING ONGMOO LEPCHA
LABZANG LEPCHA father
16
27 Female
 RBSO003384

MIMKIT LEPCHA
TASH! LEPCHA husband
16
26 Female
 RBS0041780

RINCHEN LEPCHA
LABZANG LEPCHA father
16
24 Female
 RBS0003376

SONAM LHAMU LEPCHA
LHAPZANG LEPCHA father
16
23 Female
260 RBSO052365

SANGOUP LEPCHA
TSHERING LEPCHA father
16
44 Male
 

TSERING ONGMOO LEPCHA
ONGRUP LEPCHA father
794
25 Female
 RBSO0G0533

page done 16.jpg
JANUK| GURUNG
TASH! YANGOUP LEPCHA husband
199
34 Female
 RBS0053371

NETUK LEPCHA
LENDUP LEPCHA father
196
458 Male
 FKDOO74989

CHUNK! LEPCHA
NETUK LEPCHA husband
79
55 Female
 FKDOOTSO02

ONG THINLEY LEPCHA
ONGRUP LEPCHA father
189
8 Male
409 FKDOOS5TO42

TASH! TSHERING LEPCHA
NATUK LEPCHA father
19
31 Male
 RBS0067280

PEMA CHUDEM LEPCHA
NATUR LEPCHA father
199
26 Female
 RBS0067249

TIL BOR. GURUNG
- NAR BOR. GURUNG father
2
454 Male
412 FKDO066977

LAKPA TSETEN BHUTIA
- PASSANG LHAMU BHUTIA father
2
24 Male
 RBS0049247

SONAM JIGMEE BHUTIA
SONAM DORJEE BHUTIA father
2017
29 Male
 FKDO107219

PASSANG LHAMU BHUTIA
PEM TSH. BHUTLA husband
202
54 Female
41 FKDO057349

TASH! LHAMU BHUTIA
PEM TSH. BHUTIA father
202
432 Female
 FKDO057729

NIMA CHODEN BHUTIA
PEM TSHERING BHUTIA father
202
26 Female
 RBS0041913

TASHI BHUTIA
RINZING PHUTI BHUTIA mother
203
48 
 FKDOO56655

TSHERING DOMA BHUTIA
TASHI BH

JANGMU BHUTLA
SONAM WANGD! BHUTIA husband
263
63 Female
 FKDOO73767

YANGCHEN DOMA BHUTLA
SONAM KINGA BHUTIA husband
285
44 Female
 RBSOOT0367

SONAM KINGA BHUTIA
SONAM WANGD! BHUTIA father
283
38 Male
 FKDO059220

KARZANG BHUTIA
SONAM ONGODI BHUTIA father
283
[] Male
 FKDOO56556

page done 21.jpg
NORBU THENDUP BHUTIA
SONAM \WANGOI BHUTIA father
283
34 Male
 FREDOOS9204

TENZING WANGOUP BHUTIA
SONAM WANGD) BHUTIA father
283
30 Male
 FKDO056622

PEMBA TSHERING BHUTIA
SOMAN WANGDI BHUTLA father
283
28 Male
 RBS0042143

TORGAY BHUTIA
PHURCHUANG BHUTIA father
2
40 Male
44 FKDO058453

UWEY BHUTLA
PHURCHONG BHUTLA father
2784
37 Female
 FKDO112565

WONG YAL BHUTIA
PHURGHUNG BHUTLA father
284
33 Male
 FKD0112557

BAMCHUNG BHTULA
KADOK BHUTIA father
285
42 Male
4 FKDOO67884

MACH! BHUTIA
DUKAY BHUTIA husband
285
61 Female
 FKD0067520

RINCHEN PEMA BHUTIA
KADOK BHUTIA father
2685
42 Male
 FKD0058602

PHURBA TSHERING BHUTIA
BAICHUNG BHUTIA father
285
26 Male
250 RBSO053199

PASSANG BHUTIA
RINCHE

TASH! WANGYAL BHUTIA
HARCHUNG BHUTIA father
139
26 Male
 RBSOO41673

page done 26.jpg
TSHERING PHUTI SHERPA
WARBL SHERPA father
139
34 Female
 RBS0041236

TASH! BHUTIA
NORZANG BHUTIA father
140
54 Male
 $K01/001/000397

LABCHING KI BHUTILA
LAR PA BHUTIA husband
140
45 Female
 FKOO058784

EDEN BHUTIA
BIRKHA BAHADUR SUBBAILII husband
140
46 Female
669 5K01/001/0000396

BISHNU MAYA SUBBA
RAM SINGH SUBBA husband
17
35 Female
 RBS0002659

RUPA GURUNG
HARKA BOR. GURUAG husband
18
62 Female
 FKOOOTST62

DHAN BIR GURUNG
HARKA BDR. GURUNG father
18
45 Male
 FKDO0112615

NANDU RAM GURUNG
HARKA BAHADUR GURUNG father
18
33 Male
 FKD0066175

BEN KUMAR GURUNG
- HARKA BOR GURUNG father
18
2 Male
 RBS0002782

TER BDR. GURUNG
CHANDRA BDR. GURUNG father
18
68 Male
675 FKDOOT6141

TARA DEVI GURUNG
TEK BOR. GURUNG husband
19
455 Female
 sk01/001/000053

PURNA BAHADUR GURUNG
TEK BAHADUR GURUNG father
19
49 Male
 RBSO0GG191

SABITRA GURUNG
TEK BOR GURUNG father
13
30 Female
 FKDO1Ta62

PRITAM GURUNG
TER BAH

- DAL BOR. SUBBA
HARKAMAN SUBBA father

58 Male
 FKDOOT6992

NIR MAYA SUBBA
DAL BOR. SUBBA husband
54
46 Female
 SK01/001/000151

SUK LALL SUBBA
DAL BOR SUBBA father
54
25 Male
 RBSoOO00s392

MANGAL HANG LIMBOO
LT. SANCHAMAN LIMBOO father
45
23 Male
04 RBSOOGT272

PHIPA HANG LIMBOCYSUBBA)
- SANCHAMAN SUBBA (LIMBO) father
55
233 Male
 RBSODEOESI

SANCHAMATA LIMBL
SANCHAMAN LIMBU husband
55
35 Female
 FKDO057216

SANCHAMAN SUBBA/LIMBL
(HAN DHOJ LIMBU father
55
44 Male
07 FKDOO7S770

PURNA BDR. LIMBU
DHAN DHOJ LIMBLU father
56
458 Male
 FKD0074658

ELENA RAI (SUBBA)
PRITAM LIMBOO husband
56
42 Female
 RBSOO60566

MAITA MAYA SUBBA
PURNABDOR SUBBA father
56
34 Female
 FKDOO66746

AITA LALL SUBBA
PURNA BOR SUBBA father
56
31 Male
 FKDO066738

SARITA SUBBA
PURNA BAHADUR SUBBA father
56
26 Female
 RBS0052316

MAN BIR CHETTRI
DIK BIR CHETTRI father
60
57 
 FKDOOT3924

DURGA MAYA CHETTRI
MAN BIR CHETTRI husband
60
450 Female
 FKDOOT5184

SUBASH CHETTRI
MAN BIR CHETTRI father
60
1 Male
 RBSooos07

In [ ]:
#combine all state files into one csv
combine_all_csvs("sikkim_final.csv",PARSE_DATA_CSVS)